# Coleta dos dados via webscraping

Este notebook documenta a raspagem feita no site da [CBF](https://www.cbf.com.br/) para coleta dos resultados das partidas de duas das principais competições nacionais do futebol masculino, o Campeonato Brasileiro e a Copa do Brasil, nas temporadas de 2012 a 2022.

Em todo o site, as páginas que dão acesso aos detalhes de cada partida têm o mesmo padrão: **cbf.com.br/futebol-brasileiro/competicoes/`{competicao}`/`{temporada}`/`{id_partida}`?ref=linha**

- Para os propósitos do trabalho, o parâmetro **competicao** pode assumir os valores `campeonato-brasileiro-serie-a` ou `copa-brasil-masculino`
- O parâmetro **temporada** pode assumir qualquer valor numérico entre `2012` e `2022`
- O parâmetro **id_partida** também é um valor numérico crescente, de acordo com a ordem de realização dos jogos, assumindo valores entre `1` e o número máximo de partidas previstos para a competição e temporada correspondentes.

A raspagem será feita em duas etapas. Na primeira, vamos iterar em um loop para criar todas as combinações de competição e temporada na URL base e, para cada URL criada, extrair os links das partidas. Na segunda, visitamos a página de cada partida para obter os dados de que precisamos.

## Setup

In [1]:
import datetime
import json
import re

from bs4 import BeautifulSoup
import requests as req

## Raspagem preliminar: obtendo o total de partidas por competição/temporada

Nesta seção, coletamos, para cada edição de uma das competições, a quantidade total de partidas disputadas.

Ao visitarmos a página inicial de cada competição/edição, coletaremos qualquer URL correspondente a uma partida e identificaremos aquela que possui o parâmetro `id_partida` com o maior valor, chegando, assim, ao número máximo de partidas.

In [2]:
# função auxiliar para identificação dos links
# que levam às páginas das partidas

def filtra_links_partidas(href):
    """
        Checa se o valor do atributo href de um elemento HTML
        termina com o a expressão 'ref=linha'
    """
    return href and re.compile('ref=linha$').search(href)

In [3]:
# raspagem

numero_partidas = {
    'campeonato-brasileiro-serie-a': {},
    'copa-brasil-masculino': {}
}

for competicao in numero_partidas.keys():

    for temporada in range(2012, 2023):
        
        url_competicao = f'https://www.cbf.com.br/futebol-brasileiro/competicoes/{competicao}/{temporada}'
        res = req.get(url_competicao)
        soup = BeautifulSoup(res.text)

        ids_partida = []
        # usando a função auxiliar de filtro:
        for a in soup.find_all(href=filtra_links_partidas):
            href = a['href']
            id_partida = href.split('/')[-1].split('?')[0]
            ids_partida.append(int(id_partida))

        numero_partidas[competicao][temporada] = max(ids_partida)

In [4]:
# o resultado da célula anterior é um dicionário contento
# o total de partidas em cada edição de cada competição
# (não era uma checagem necessária para o Brasileirão,
# pois a quantidade de equipes é a mesma de 2012 para cá)
print(json.dumps(numero_partidas, indent=4))

{
    "campeonato-brasileiro-serie-a": {
        "2012": 380,
        "2013": 380,
        "2014": 380,
        "2015": 380,
        "2016": 380,
        "2017": 380,
        "2018": 380,
        "2019": 380,
        "2020": 380,
        "2021": 380,
        "2022": 380
    },
    "copa-brasil-masculino": {
        "2012": 126,
        "2013": 172,
        "2014": 172,
        "2015": 172,
        "2016": 170,
        "2017": 120,
        "2018": 120,
        "2019": 120,
        "2020": 120,
        "2021": 122,
        "2022": 108
    }
}


## Raspagem principal

Nesta seção, definimos, primeiro, a função `scrap_partida()`, que será responsável por extrair de cada página os dados de interesse: equipes, placares (do tempo normal de jogo e das disputas de pênaltis) e jogadores que marcaram os gols da partida.

Em seguida, definimos um laço para iterar sobre a sequência de partidas de cada competição/temporada e aplicar a função criada. Uma observação importante deve ser feita em relação à temporada 2022, que ainda não se encontra encerrada no momento de escrita deste notebook (quando estavam realizadas as partidas de número 160 do Campeonato Brasileiro e 108 da Copa do Brasil).

In [5]:
# função reponsável pela raspagem principal

def scrap_partida(url):
    """
        Acessa a página da partida e coleta os dados principais
        - local, data e hora
        - placar do tempo normal de jogo
        - placar de eventual disputa de pênaltis
        - jogadores que marcaram gol
    """
    res = req.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    dados = [competicao, temporada, id_partida]

    # local, data e hora
    agenda = [span.text.strip() for span in soup.find_all('span', class_='text-2 p-r-20')][:3]
    dados.extend(agenda)

    # placar, nome e jogadores da equipe mandante que marcaram gols
    mandante = re.split('\n+', soup.find('div', class_='time-left').text)[1:-1]
    if len(mandante) < 2:
        mandante = [0] + mandante
    dados.extend(mandante[:2])
    dados.append(sep.join(mandante[2:]))

    # placar, nome e jogadores da equipe visitante que marcaram gols
    visitante = re.split('\n+', soup.find('div', class_='time-right').text)[1:-1]
    if len(visitante) < 2:
        mandante = [0] + visitante
    dados.extend(visitante[:2])
    dados.append(sep.join(visitante[2:]))

    # identifica gols contra (dentre os já coletados anteriormente)
    gols_contra = set([p.text for p in soup.find_all('p', class_='color-red')])
    dados.append(sep.join(gols_contra))

    # placar da disputa de pênaltis
    penaltis = [0, 0]
    try:
        disputa_penaltis = re.split('\n+', soup.find('div', class_='x center-block').text)
        if len(disputa_penaltis)>2:
            penaltis = disputa_penaltis[2:4]
    except AttributeError:
        penaltis = [0, 0]    
    dados.extend(penaltis)
    
    # agrupa e retorna todos os dados em uma lista
    dados = [str(x) for x in dados]    
    return dados

In [6]:
# iterando sobre as urls de cada página
# para aplicar a função de raspagem

sep = ' |'
arquivo_resultados = 'dados/raw-scrap.csv'
erros = []

for competicao, temporadas in numero_partidas.items():
    print(competicao, end='')
    
    for temporada, max_partidas in temporadas.items():
        print(f'\n\ttemporada {temporada}:', end='')
        
        if str(temporada)=='2022':
            if competicao=='campeonato-brasileiro-serie-a':
                max_range = 160 + 1  # até o final da 16ª rodada
            else:
                max_range = 108 + 1  # até oitavas-de-final
        else:
            max_range = max_partidas + 1
        
        for id_partida in range(1, max_range):
            
            url = f'https://www.cbf.com.br/futebol-brasileiro/competicoes/{competicao}/{temporada}/{id_partida}'
            
            # faz a raspagem e salva os resultados num arq. csv
            try:
                dados = scrap_partida(url)
                linha = ','.join(dados)
                linha += '\n'
                with open(arquivo_resultados, 'a', encoding='utf8') as f:
                    f.write(linha)
            except Exception as e:
                erro = (competicao, temporada, id_partida, str(e))
                erros.append(erro)
            finally:
                loader = int(id_partida/max_range*100)
                print('\r\ttemporada %s: partida %03d [%02d%%]' % (temporada, id_partida, loader), end='')
    
    print()

campeonato-brasileiro-serie-a
	temporada 2012: partida 380 [100%]
	temporada 2013: partida 380 [100%]
	temporada 2014: partida 380 [100%]
	temporada 2015: partida 380 [100%]
	temporada 2016: partida 380 [100%]
	temporada 2017: partida 380 [100%]
	temporada 2018: partida 380 [100%]
	temporada 2019: partida 380 [100%]
	temporada 2020: partida 380 [100%]
	temporada 2021: partida 380 [100%]
	temporada 2022: partida 160 [100%]
copa-brasil-masculino
	temporada 2012: partida 126 [100%]
	temporada 2013: partida 172 [100%]
	temporada 2014: partida 172 [100%]
	temporada 2015: partida 172 [100%]
	temporada 2016: partida 170 [100%]
	temporada 2017: partida 120 [100%]
	temporada 2018: partida 120 [100%]
	temporada 2019: partida 120 [100%]
	temporada 2020: partida 120 [100%]
	temporada 2021: partida 122 [100%]
	temporada 2022: partida 108 [100%]


In [7]:
# verifica se algum erro foi armazenado
erros

[]